# Distilling Knowledge in Multiple Students Using Generative Models

In [1]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(teacher.summary())

# Train the teacher model as usual
# epochs = 0
# batch_size = 256
# teacher.fit(X_train, Y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(X_val, Y_val))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [3]:
teacher.load_weights("Teacher_FMNIST_92.h5")

In [4]:
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
X_test /= 255


teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 3s 4ms/step - loss: 0.3734 - accuracy: 0.9195


[0.351641982793808, 0.9239000082015991]

In [5]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)


# 4 Students case
# ---------------------------------------------
s1Train=train_dense[:,:64]
s2Train=train_dense[:,64:128]
s3Train=train_dense[:,128:192]
s4Train=train_dense[:,192:]

s1Val=val_dense[:,:64]
s2Val=val_dense[:,64:128]
s3Val=val_dense[:,128:192]
s4Val=val_dense[:,192:]


In [6]:
def define_model(name):

    student1 = Sequential()
    student1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(16, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(64,name='req'+name))

    student1.compile('adam',loss='mean_squared_error',metrics=['accuracy'])

    return student1



In [7]:
s1=define_model("s1")
s2=define_model('s2')
s3=define_model("s3")
s4=define_model('s4')

In [9]:
batch_size = 128
s1.fit([X_train], s1Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s1Val))
s2.fit([X_train], s2Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s2Val))
s3.fit([X_train], s3Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s3Val))
s4.fit([X_train], s4Train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val], s4Val))

Epoch 1/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3570 - accuracy: 0.4154 - val_loss: 0.2394 - val_accuracy: 0.4706
Epoch 2/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3498 - accuracy: 0.4188 - val_loss: 0.2353 - val_accuracy: 0.4794
Epoch 3/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3444 - accuracy: 0.4211 - val_loss: 0.2348 - val_accuracy: 0.4757
Epoch 4/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3457 - accuracy: 0.4156 - val_loss: 0.2338 - val_accuracy: 0.4730
Epoch 5/10
375/375 [==============================] - 4s 10ms/step - loss: 0.3419 - accuracy: 0.4170 - val_loss: 0.2153 - val_accuracy: 0.4862
Epoch 6/10
375/375 [==============================] - 2s 6ms/step - loss: 0.3388 - accuracy: 0.4197 - val_loss: 0.2353 - val_accuracy: 0.4737
Epoch 7/10
375/375 [==============================] - 3s 7ms/step - loss: 0.3366 - accuracy: 0.4191 - val_loss: 0.2072 - val_accuracy: 0.4901
Epoch

### **4 Student**

In [10]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
o3=s3.get_layer("reqs3").output
o4=s4.get_layer("reqs4").output
output=tensorflow.keras.layers.concatenate([o1,o2,o3,o4])
print (output.shape)
output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)
mm4=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,s3.get_layer("s3").input,s4.get_layer("s4").input],output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm4.get_layer('d1').set_weights(my_weights)

(None, 256)


In [11]:
for l in mm4.layers[:len(mm4.layers)-1]:
    l.trainable=False

mm4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [12]:
batch_size = 128
mm2_history=mm4.fit([X_train,X_train,X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=1, validation_data=([X_val,X_val,X_val,X_val], Y_val))

Epoch 1/20
375/375 [==============================] - 4s 10ms/step - loss: 0.6471 - accuracy: 0.7953 - val_loss: 0.4045 - val_accuracy: 0.8551
Epoch 2/20
375/375 [==============================] - 4s 10ms/step - loss: 0.5446 - accuracy: 0.7979 - val_loss: 0.3957 - val_accuracy: 0.8532
Epoch 3/20
375/375 [==============================] - 2s 6ms/step - loss: 0.5031 - accuracy: 0.8074 - val_loss: 0.3935 - val_accuracy: 0.8532
Epoch 4/20
375/375 [==============================] - 4s 10ms/step - loss: 0.5030 - accuracy: 0.8041 - val_loss: 0.3976 - val_accuracy: 0.8429
Epoch 5/20
375/375 [==============================] - 4s 11ms/step - loss: 0.4829 - accuracy: 0.8084 - val_loss: 0.3947 - val_accuracy: 0.8513
Epoch 6/20
375/375 [==============================] - 5s 12ms/step - loss: 0.4822 - accuracy: 0.8095 - val_loss: 0.3967 - val_accuracy: 0.8489
Epoch 7/20
375/375 [==============================] - 4s 10ms/step - loss: 0.4750 - accuracy: 0.8122 - val_loss: 0.3953 - val_accuracy: 0.8483


In [14]:
loss, acc = mm4.evaluate([X_test,X_test,X_test,X_test], Y_test, verbose=1)
loss, acc 

313/313 [==============================] - 3s 8ms/step - loss: 0.4038 - accuracy: 0.8527


(0.4038444757461548, 0.8526999950408936)